# Worksheet 6 example code

## Example 1 Frequency of warm days (TX90P) in the future

Calculate the baseline (1986-2005) 90th percentile of maximum temperature. Then calculate the frequency of warm days in the future (2041-2060). Do this for HadGEM2-ES and MPI-ESM-LR.

In [ ]:
# Code preamble - these libraries will be used in this worksheet.
# This code block needs to be re-run every time you restart this worksheet!
%matplotlib inline 
import os
import iris
import iris.coord_categorisation
import iris.quickplot as qplt
import iris.plot as iplt
from iris.util import equalise_attributes
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
from iris.analysis import Aggregator

# Some helpful data locations
DATADIR = 'data_v2'
CHIRPSDIR = os.path.join(DATADIR, 'CHIRPS')
CRUDIR = os.path.join(DATADIR, 'CRU')
CLIMDIR = os.path.join(DATADIR, 'EAS-22', 'climatology/')
MODELDIR = os.path.join(DATADIR, 'EAS-22/')
GCMIDS = ['hadgem2-es', 'mpi-esm-lr']
GCM_FULL = {'hadgem2-es':'MOHC-HadGEM2-ES' , 'mpi-esm-lr':'MPI-M-MPI-ESM-LR'}
TIME_PERIODS = {'historical':'1986_2005', 'rcp85':'2041_2060'}

First calculate the 90th percentile of tsmax in the baseline and future periods for both set of downscaled GCMs.

In [ ]:
# Define a helper function to extract our cube chunks
def chunks(cube, x=200, y=200):
    """
    Yield successive x-y sized chunks from cube,
    works for 3D Time-Lat-Lon
    
    Args:
    cube (Iris cube): input cube to be chunked
    x: size of chunks in x direction
    y: size of chunk in y direction
    """
    coord_names = [coord.name() for coord in cube.coords()]
    if 'grid_latitude' in coord_names and 'latitude' in coord_names:
        cube.remove_coord('latitude')
    if 'grid_longitude' in coord_names and 'longitude' in coord_names:
        cube.remove_coord('longitude')
    
    for i in range(0, cube.coord(axis='x').shape[0], x):
        for j in range(0, cube.coord(axis='y').shape[0], y):
            yield cube[:, j:j + y, i:i + x]

    
def chunks_2d(cube, x=200, y=200):
    coord_names = [coord.name() for coord in cube.coords()]
    if 'grid_latitude' in coord_names and 'latitude' in coord_names:
        cube.remove_coord('latitude')
    if 'grid_longitude' in coord_names and 'longitude' in coord_names:
        cube.remove_coord('longitude')
    
    for i in range(0, cube.coord(axis='x').shape[0], x):
        for j in range(0, cube.coord(axis='y').shape[0], y):
            yield cube[j:j + y, i:i + x]
    

In [ ]:
for gcmid in GCMIDS:
    for period in TIME_PERIODS.keys():
        filename = f'tasmax_EAS-22_{GCM_FULL[gcmid]}_{period}_*GERICS-REMO2015*_*_day_*.nc'
        model_tasmax = iris.load(MODELDIR + filename)
        # solve merge issues
        equalise_attributes(model_tasmax)
        model_tasmax = model_tasmax.concatenate_cube()

        # 90th percentile calculation
        model_pc90 = iris.cube.CubeList()
        for model_data in chunks(model_tasmax):
            model_pc90_chk = model_data.collapsed('time', iris.analysis.PERCENTILE, percent=90.0)
            model_pc90.append(model_pc90_chk)
        model_pc90 =model_pc90.concatenate_cube()

        # save to file
        outfile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS[period] + '.GERICS-REMO2015.tasmax.90pc.nc')
        print(f'saving to file: {outfile}')
        iris.save(model_pc90, outfile)
        

Calculate the frequency of warm days in the future (extreme index TX90P), i.e. the number of days which exceed the 90th percentile temperatures in the baseline.  Then calculate the numbers of days as a percentage.


In [ ]:
for gcmid in GCMIDS:
    
    # load daily data for future 
    filename = f'tasmax_EAS-22_{GCM_FULL[gcmid]}_rcp85_*GERICS-REMO2015*_*_day_*.nc'
    future_tasmax = iris.load(MODELDIR + filename)

    # solve merge issues
    equalise_attributes(future_tasmax)
    future_tasmax = future_tasmax.concatenate_cube()  
    ndays = len(future_tasmax.coord('time').points)
    
    infile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.tasmax.90pc.nc')
    print(infile)
    hist_90pc = iris.load_cube(infile)
    
    
    # need to do next operation chunking data, also chunk 90th percentile 
    nwarmdays_future = iris.cube.CubeList()  
    for chunk_ft_tasmax, chunk_hist_90pc in zip(chunks(future_tasmax), chunks_2d(hist_90pc)):

        # Use np.where to identify all cells where daily temperatures 
        # in the future exceed the 90th percentile
        assert(chunk_ft_tasmax.coord('grid_longitude') == chunk_hist_90pc.coord('grid_longitude'))
        assert(chunk_ft_tasmax.coord('grid_latitude') == chunk_hist_90pc.coord('grid_latitude'))
        temp_gt_chunk = np.where(chunk_ft_tasmax.data >= chunk_hist_90pc.data, 1, 0)
              
        # use the 90th percentile cube as a template to copy warm days data into 
        nwarmdays_future_chunk = chunk_hist_90pc.copy()
        nwarmdays_future_chunk.data = np.ma.sum(temp_gt_chunk.data, axis=0)
        nwarmdays_future.append(nwarmdays_future_chunk)
        
    nwarmdays_future = nwarmdays_future.concatenate_cube()

    # the sum above removes the mask - reinstate it with 
    nwarmdays_future.data.mask = hist_90pc.data.mask
    nwarmdays_future.units = '1'
    nwarmdays_future.rename('days > 90th %ile of baseline ')
      
    print ("Saving numbers of warm days in the future from " + gcmid)
    outfile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.tasmax.nwarmdays.nc')
    iris.save(nwarmdays_future, outfile)
    
    # calculate percentage of days (see below)
    nwarm_pc = nwarmdays_future/ndays*100.
    nwarm_pc.units = '%'
    nwarm_pc.rename('percentage of days T > 90th %ile of baseline ')
    
    # save percentage
    print ("Saving precentage of warm days in the future from " + gcmid)
    outfile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.tasmax.nwarmpc.nc')    
    iris.save(nwarm_pc, outfile)


Plot the numbers of warm days in the future and the percentage of warm days.

In [ ]:
plt.figure(figsize=(12, 12))

for i, gcmid in enumerate(GCMIDS):
    
    infile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.tasmax.nwarmdays.nc')
    nwarmdays = iris.load_cube(infile)
    infile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.tasmax.nwarmpc.nc')    
    nwd_pcent = iris.load_cube(infile)

    plotnum = 1 + 2*i
    plt.subplot(2, 2, plotnum)
    qplt.pcolormesh(nwarmdays, vmin=0, vmax=10000)
    plt.title(gcmid + ': Number of warm days')
    plt.gca().coastlines()
    plt.subplot(2, 2, plotnum+1)
    qplt.pcolormesh(nwd_pcent, vmin=0, vmax=30)
    plt.title(gcmid + ': Percentage of warm days')
    plt.gca().coastlines()

plt.show()

## Example 2. Percentage of total precipitation which falls on very wet days

Calculate the percentage of total precipitation which falls on very wet days in the future over Thailand
(where a very wet day is one on which daily rainfall exceeds the 95th percentile of the baseline).


First find the 95th percentile of rainfall during baseline

In [ ]:
for gcmid in GCMIDS:
    filename = f'pr_EAS-22_{GCM_FULL[gcmid]}_historical_*GERICS-REMO2015*_*_day_*.nc'
    model_pr = iris.load(MODELDIR + filename)
    # solve merge issues
    equalise_attributes(model_pr)
    model_pr = model_pr.concatenate_cube()

    # 95th percentile calculation
    model_pc95 = iris.cube.CubeList()
    for model_data in chunks(model_pr):
        model_pc95_chk = model_data.collapsed('time', iris.analysis.PERCENTILE, percent=95.0)
        model_pc95.append(model_pc95_chk)
    model_pc95 = model_pc95.concatenate_cube()

    # save to file
    outfile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.pr.95pc.nc')
    print(f'saving to file: {outfile}')
    iris.save(model_pc95, outfile)


In [ ]:
# Coordinates of a box around Thailand
thai_lons = np.array([98.0, 98.0, 105.0, 105.0])
thai_lats = np.array([10.0, 21.0,  10.0,  21.0])

# Load a cube on the rotated grid
gcmid = 'hadgem2-es'
infile = os.path.join(CLIMDIR, gcmid +  '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.pr.95pc.nc')
rotg = iris.load_cube(infile)
rcs = rotg.coord('grid_latitude').coord_system

# Get the rotated pole coordinates
pole_lat = rcs.grid_north_pole_latitude
pole_lon = rcs.grid_north_pole_longitude

# Convert the coordinates of a box around Thailand from real coordinates to rotated polar coordinates
grid_lons, grid_lats = iris.analysis.cartography.rotate_pole(thai_lons, thai_lats, pole_lon, pole_lat)

# Find the max / min of the lons / lats on the rotated grid.  They will be used to extract the data around Malaysia
# N.B. The conversion to float is needed, as numpy data are of type float64 by default. If the coordinate limits
# are passed as float64, they are interpreted as a list of two floats and the program will stop with an error:
# ValueError: setting an array element with a sequence.
lon_0 = float(min(grid_lons))
lon_1 = float(max(grid_lons))
lat_0 = float(min(grid_lats))
lat_1 = float(max(grid_lats))

# Set up constraints on the rotated grid for Thailand
lon_con = iris.Constraint(grid_longitude = lambda cell: lon_0 <= cell <= lon_1)
lat_con = iris.Constraint(grid_latitude = lambda cell: lat_0 <= cell <= lat_1)


In [ ]:
# loop over GCMs
for i, gcmid in enumerate(GCMIDS):
    # load the RCP 8.5 daily precip data
    file_f =  f'pr_EAS-22_{GCM_FULL[gcmid]}*_rcp85_*_GERICS-REMO2015*_*_day_*.nc'
#   precip = iris.load_cube(data_path + file_f, lon_con).intersection(grid_latitude = (-14.767, -5,623))
    precip = iris.load(MODELDIR + file_f, lat_con & lon_con)
    # solve merge issues
    equalise_attributes(precip)
    precip = precip.concatenate_cube()
   
    
    # load the historical 95th percentile
    file_f = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.pr.95pc.nc')
    precip_pc95 = iris.load_cube(file_f, lat_con & lon_con)
    
# Use broadcasting to identify all cells in precip where p95 is exceeded
    pre_gt_pc95 = np.where(precip.data > precip_pc95.data, precip.data, 0.0)
    pre_p95 = np.sum(pre_gt_pc95, axis=0)
    pre_tot = precip.collapsed('time', iris.analysis.SUM)
    pre_tot.data = np.divide(pre_p95, pre_tot.data, out=np.zeros_like(pre_tot.data), where = pre_tot.data != 0)
    pre_tot = iris.analysis.maths.multiply(pre_tot, 100)
    file_out = gcmid + '.R95pTOT.future.GERICS-REMO2015.nc'
    iris.save(pre_tot, CLIMDIR + file_out)

Plot the percentages of heavy rainfall in the future.

In [ ]:
plt.figure(figsize=(12, 6))

for i, gcmid in enumerate(GCMIDS):
    filename = gcmid + '.R95pTOT.future.GERICS-REMO2015.nc'
    pcent_heavy_rain = iris.load_cube(CLIMDIR + filename)
    plotnum = 1 + i
    plt.subplot(1, 2, plotnum)
    qplt.pcolormesh(pcent_heavy_rain, vmin=0, vmax=100)
    plt.title(gcmid + ': Percentage of heavy rain \n over Thailand in the future')
    plt.gca().coastlines()

plt.show()

<p><img src="img/MO_MASTER_black_mono_for_light_backg_RBG.png" alt="Met Office Logo" style="float: center; height: 100px;"/></p>
<center>© Crown Copyright, Met Office. All rights reserved.</center>
<center>This file is part of PyPrecis and is released under the BSD 3-Clause license.</center>
<center>See LICENSE in the root of the repository for full licensing details.</center>